In [26]:
import os 
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
import gradio as gr 

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
# Load environment variables from .env
load_dotenv()

# Define your API keys and expected prefixes
api_keys = {
    "OPEN_AI_API_KEY": "sk-proj-",
    "ANTHROPIC_API_KEY": "sk-ant-",
    "GOOGLE_GEMINI_API_KEY": "gems-"
}

def validate_key(name, prefix):
    key = os.getenv(name)
    if key and key.startswith(prefix):
        print(f"✅ {name} is valid.")
    elif key:
        print(f"⚠️ {name} is present but may be malformed.")
    else:
        print(f"❌ {name} is missing from environment.")
    return key

# Validate and assign keys
openai_key = validate_key("OPEN_AI_API_KEY", api_keys["OPEN_AI_API_KEY"])
anthropic_key = validate_key("ANTHROPIC_API_KEY", api_keys["ANTHROPIC_API_KEY"])
gemini_key = validate_key("GOOGLE_GEMINI_API_KEY", api_keys["GOOGLE_GEMINI_API_KEY"])

# Assign keys to libraries or environment if needed
if openai_key:
    import openai
    openai.api_key = openai_key

if anthropic_key:
    from anthropic import Anthropic
    print("Anthropic SDK loaded successfully")
    # client = Anthropic.Client(api_key=anthropic_key)
    client = Anthropic(api_key=anthropic_key)


if gemini_key:
    import google.generativeai as genai
    genai.configure(api_key=gemini_key)

    os.environ["GOOGLE_GEMINI_API_KEY"] = gemini_key

✅ OPEN_AI_API_KEY is valid.
✅ ANTHROPIC_API_KEY is valid.
⚠️ GOOGLE_GEMINI_API_KEY is present but may be malformed.
Anthropic SDK loaded successfully


In [28]:
system_message = "You are helpful assistant"

In [29]:
def message_gpt(prompt):
    messages = [{"role":"system", "content":system_message},{"role":"user", "content":prompt}]
    completion = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return completion.choices[0].message.content

In [30]:
message_gpt("what is today's date?")

"Today's date is October 5, 2023."

In [31]:
def shout(text):
    print(f"shout has been call with input {text}")
    return text.upper()

In [32]:
shout("Hello World!")

shout has been call with input Hello World!


'HELLO WORLD!'

In [33]:
gr.Interface(fn=shout, inputs="textbox",allow_flagging="never", outputs="textbox").launch(share=True)

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://699b940a840f96d24b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [34]:
view = gr.Interface(
fn=shout,
inputs=[gr.Textbox(label="Your message:", lines=6)],
outputs=[gr.Textbox(label="Response:", lines=8)],
allow_flagging="never"
)
view.launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [35]:
view = gr.Interface(
fn=shout,
inputs=[gr.Textbox(label="Your message:", lines=6)],
outputs=[gr.Textbox(label="Response:", lines=8)],
allow_flagging="never"
)
view.launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [36]:
system_message = "You are a helpful assistant that respond in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your Message:", lines=6)],
    outputs=[gr.Textbox(label="Your Response:", lines=6)]
)   
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [37]:
system_message = "You are a helpful assistant that respond in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Your Response:")],
    allow_flagging="never"
)
view.launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [38]:
def stream_gpt(prompt):
    messages = [
        {"role":"system", "content":system_message},
        {"role":"user", "content":prompt}
    ]
    stream = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [39]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message")],
    outputs=[gr.Markdown(label="Response")],
    allow_flagging="never"
)
view.launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [40]:
def stream_claude(prompt):
    result = client.messages.stream(
        model="claude-3-5-haiku-latest",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt}
        ])
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response
    

In [41]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your Message")],
    outputs=[gr.Markdown(label="Response")]
)
view.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [42]:
def stream_model(prompt,model):
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [43]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your Message:"),gr.Dropdown(["GPT", "Claude"], label="Select Model")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
    )
view.launch()

c:\Users\mathe\OneDrive\Desktop\LLM_Course\Large_Language_Model_course_Material\Week2\.venv\lib\site-packages\gradio\interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [49]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [56]:
system_prompt = "You are an assistant that analyzes the content of a company website landing page \
and create a short brochure about the company for prospective customers, investors and recruits, Respond in markdown with some jokes to make it fun to read."

In [57]:
def stream_brochure(company_name,url,model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page"
    prompt+= Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown Model")
    
    for chunk in result:
        yield chunk  # yield each string chunk, not the generator
        # yield result


In [55]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[gr.Textbox(label="Company Name:"), 
            gr.Textbox(label="Landing page URL including http:// or https://"),
            gr.Dropdown(["GPT", "Claude"], label="Select Model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
